In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import *
import numpy as np
from sklearn.cluster import KMeans

In [4]:
config = {"dataset": "blog", "topics": "lda"}
data_train, data_val, data_test = load_data(config)
x = [data_test[i]['sentence'] for i in range(100)]
# x = [xi for xi in x if len(xi.split()) > 1]
# topics, word2idx = get_topics(config, x)

Found cached dataset blog_authorship_corpus (/home/steinad/.cache/huggingface/datasets/blog_authorship_corpus/blog_authorship_corpus/1.0.0/6f5d78241afd8313111956f877a57db7a0e9fc6718255dc85df0928197feb683)
Found cached dataset blog_authorship_corpus (/home/steinad/.cache/huggingface/datasets/blog_authorship_corpus/blog_authorship_corpus/1.0.0/6f5d78241afd8313111956f877a57db7a0e9fc6718255dc85df0928197feb683)
Loading cached processed dataset at /home/steinad/.cache/huggingface/datasets/blog_authorship_corpus/blog_authorship_corpus/1.0.0/6f5d78241afd8313111956f877a57db7a0e9fc6718255dc85df0928197feb683/cache-91bee97bd74f5a2a.arrow
Loading cached processed dataset at /home/steinad/.cache/huggingface/datasets/blog_authorship_corpus/blog_authorship_corpus/1.0.0/6f5d78241afd8313111956f877a57db7a0e9fc6718255dc85df0928197feb683/cache-fabb96a2d990d2e2.arrow


In [6]:
dataset = "blog"
config = {"dataset": dataset, "topics": "lda"}
topics, word2idx = get_topics(config, x)
# roberta_shap_vals = load("shap_vals_gpt2_sst2")
roberta_topic_vals = load(f"topic_vals_distilroberta_{dataset}_lda").reshape(-1, 31, 5)
roberta_word_vals = load(f"word_vals_distilroberta_{dataset}_lda")
# gpt2_topic_vals = load(f"topic_vals_gpt2_{dataset}_lda").reshape(-1, 31)
# gpt2_word_vals = load(f"word_vals_gpt2_{dataset}_lda")

[[[-1.83110830e-03 -2.28541243e-03  4.76596788e-03  9.03018709e-04
   -9.72774600e-04]
  [-4.56672795e-03 -1.23948330e-02  1.90876040e-02  1.70953730e-02
   -1.76202119e-02]
  [-1.00317569e-03 -1.06399252e-03  2.31555756e-03  2.07160924e-03
   -2.09181920e-03]
  ...
  [-5.13386766e-03 -5.93131461e-03  1.18152253e-02  1.36682045e-02
   -1.39388373e-02]
  [-2.10988910e-03 -1.14157639e-03  3.66272985e-03  1.75312241e-03
   -1.84189796e-03]
  [-7.43208023e-02 -1.33828489e-01  2.25261041e-01  2.54432237e-01
   -2.56774188e-01]]

 [[ 6.36622124e-03 -5.68307352e-03 -7.12220114e-04  8.31600828e-04
   -6.57964214e-04]
  [ 1.25268497e-02 -1.11453457e-02 -1.99797497e-03  3.71074187e-03
   -3.75788151e-03]
  [ 3.75723545e-03 -3.37631277e-03 -5.61349138e-04  7.91916432e-04
   -8.12950315e-04]
  ...
  [ 1.80022132e-02 -1.59178442e-02 -3.30123717e-03  3.54622948e-03
   -3.81129762e-03]
  [ 5.12343988e-03 -4.40378015e-03 -1.24138711e-03  4.00002516e-04
   -4.82361620e-04]
  [ 4.83114505e-01 -4.3480720

In [13]:
base_path = "../scripts/"
topics_matrix_df = pd.read_csv(base_path + ("../data/processed_LDA_files/" + dataset + "_50.csv"))
word2idx = dict(zip(topics_matrix_df["words"], range(len(topics_matrix_df["words"]))))
topics_matrix_df.drop(columns=["words"], inplace=True)
topics_matrix_df = topics_matrix_df.T
topics_raw = topics_matrix_df.to_numpy()
sorted_words = np.argsort(topics_raw, axis=1)
idx2word = {i: word for word, i in word2idx.items()}
for i in range(topics_raw.shape[0]):
    print(i, [idx2word[word_idx] for word_idx in sorted_words[i][::-1][:10]])

0 ['oh', 'day', 'happy', 'shit', 'fuck', 'fucking', 'man', 'haha', 'yeah', 'one']
1 ['post', 'here', 'comment', 'sub', 'read', 'sorry', 'from', 'thanks', 'got', 'reddit']
2 ['name', 'game', 'one', 'watch', 'show', 'great', 'favorite', 'play', 'fun', 'video']
3 ['name', 'we', 'game', 'our', 'team', 'him', 'play', 'win', 'would', 'season']
4 ['their', 'money', 'because', 'people', 'pay', 'we', 'our', 'or', 'government', 'want']
5 ['good', "i'm", 'sorry', 'hope', 'glad', 'luck', 'am', "you're", 'best', 'hear']
6 ['them', 'will', 'take', 'from', "don't", 'stop', 'please', 'yourself', 'get', 'or']
7 ['thanks', 'thank', 'much', ':)', 'very', "i'll", 'really', 'appreciate', 'try', 'good']
8 ['would', 'did', 'say', "didn't", 'know', 'same', 'said', 'thought', 'thing', 'were']
9 ['people', 'who', 'their', 'them', 'other', 'because', 'name', 'these', 'there', 'by']
10 ['would', 'one', 'those', 'only', 'an', 'we', 'get', 'also', 'which', 'why']
11 ['up', 'down', 'when', 'back', 'there', 'go', 'ou

In [18]:
model1, _ = load_models(config)
explainer = shap.Explainer(model1, padding="max_length", truncation=True, max_length=512)
idx = 4
tok_sample = explainer.masker.data_transform(x[idx])[0]
print(tok_sample)
_, words = word_shap(tok_sample, np.zeros(len(tok_sample)))
for tok in words:
    if tok not in word2idx:
        print(tok)
print(roberta_word_vals[idx])
print(roberta_topic_vals[idx])
print(sum(roberta_word_vals[idx]))
print(sum(roberta_topic_vals[idx]))
print(x[idx])

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

They
for
of
and
on
to
the
afterlife.
[0.2003762051463127, -0.15281378291547298, -0.20729783363640308, 0.11858675349503756, -0.08425124082714319, 0.007137363776564598, 0.01972377859055996, 0.005102875952919324, -0.004639976347486178, -0.005184094111124675, -0.008481353521347046, 0.09562041610479355, -0.0773390680551529, -0.006970260292291641, 0.0005980096757411957, 0.0005573878685633341, -0.023366148273150124]
[ 0.         -0.01104525  0.00581416 -0.01441691  0.00483219 -0.00590993
  0.01627969 -0.09985756 -0.00693229  0.00785239  0.09768289  0.
  0.          0.         -0.04073291 -0.00318874 -0.06766862  0.
  0.          0.00771972  0.00675133  0.         -0.00959804 -0.00306536
  0.         -0.00611068 -0.03786474 -0.09854012 -0.04376533  0.0013255
  0.17779764]
-0.12264096736907959
-0.12264096736907962
They got bored from haunting earth for thousands of years and ultimately moved on to the afterlife.


In [31]:
X = roberta_topic_vals / np.sum(roberta_topic_vals, axis=1, keepdims=True)
kmeans = KMeans(n_clusters=10).fit(X)
clabels = kmeans.labels_

diffs = np.zeros(10)
for c in range(10):
    diffs[c] = np.sum(np.abs(np.sum(roberta_topic_vals[clabels == c], axis=0) - np.sum(gpt2_topic_vals[clabels == c], axis=0)) / np.sum(clabels == c))
most_diff_cluster = np.argsort(diffs)[::-1][0]
print(diffs)
print(most_diff_cluster)
diff = np.abs(np.sum(roberta_topic_vals[clabels == most_diff_cluster], axis=0) - np.sum(gpt2_topic_vals[clabels == most_diff_cluster], axis=0)) / np.sum(clabels == most_diff_cluster)
print(np.argsort(diff)[::-1][:3])

for idx in np.nonzero(clabels == most_diff_cluster)[0]:
    print(idx, x[idx])

[0.35931513 0.74429793 1.18468939 0.46055958 0.3257553  1.45500909
 1.02774093 1.00907149 1.04191695 0.51910663]
5
[30 25 19]
29 here 's a british flick gleefully unconcerned with plausibility , yet just as determined to entertain you .


In [47]:
diffs = np.sum(np.abs(roberta_topic_vals - gpt2_topic_vals), axis=1)
biggest_diff = np.argsort(diffs)
print(diffs[biggest_diff[-5:]])

[1.18468939 1.24345609 1.34729508 1.45500909 1.50728886]


In [32]:
ind = 29 #biggest_diff[-1]
print(x[ind])
topic_names = np.array([f"topic_{i}" for i in range(36)])
topic_values, sorted_names = sort_shap(roberta_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

print()
topic_values, sorted_names = sort_shap(gpt2_topic_vals[ind], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

here 's a british flick gleefully unconcerned with plausibility , yet just as determined to entertain you .
topic_19: -0.1553845656310764
topic_30: -0.10612585631315596
topic_25: -0.0935087014799168
topic_14: -0.03646532557338526
topic_15: -0.01593457268374577
topic_22: 0.02807360821965514
topic_1: 0.045695628701287475
topic_21: 0.0492626937546679
topic_26: 0.09268165987427164
topic_10: 0.22321857630317818
0.06044590473175046

topic_19: -0.3872873015917917
topic_0: 0.0
topic_17: 0.0
topic_16: 0.0
topic_13: 0.0
topic_15: 0.029227123956237402
topic_4: 0.03711597906478719
topic_26: 0.05891144657342237
topic_25: 0.14575828416624173
topic_30: 0.3104190980375279
0.358201801776886


In [7]:
topic_names = np.array([f"topic_{i}" for i in range(36)])
roberta_feat_imp = np.sum(np.abs(roberta_topic_vals), axis=0)
# gpt2_feat_imp = np.sum(np.abs(gpt2_topic_vals), axis=0)

topic_values, sorted_names = sort_shap(roberta_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

# print()
# topic_values, sorted_names = sort_shap(gpt2_feat_imp, topic_names)
# for i in range(5):
#     print(f"{sorted_names[i]}: {topic_values[i]}")
# for i in range(5, 0, -1):
#     print(f"{sorted_names[-i]}: {topic_values[-i]}")

topic_25: 0.006110679304319723
topic_15: 0.006897511634863846
topic_12: 0.009089059525776176
topic_22: 0.012098819741992079
topic_21: 0.015178255319194547
topic_1: 0.2552797217033191
topic_26: 0.5835412084444859
topic_7: 0.6627277899339665
topic_5: 1.5791001013991892
topic_30: 4.3985357037551385
